In [ ]:
import utils
import os
import pandas as pd

from tensorflow.keras.models import load_model
from ipywidgets import interact

trained_models = "./trained_models"

Extract the qptiff files from the directory and puts them into a processed file inside
the directory path. In this case "./data/66-4".

This line will create a directory called "processed_XX" where XX is the selected resolution

In [ ]:
utils.resize_qptiff_files("./data/multiplex", "32")

Given the downsampled repository we will run an algorithm to remove dust and bubbles. Its output will be saved into "processed_XX" into a new directory called "clean_dust_bubbles". This was built this way to avoid overwritting files, allowing an easier resolution of errors if present.

In [ ]:
# utils.remove_dustNbubbles("./data/66-4/processed_32", image_resolution=32)

Given the path of the images you want to align this will create a list of directories with different content within it. Here are the most relevant ones:

-  "TA" image mask form which the alignment is performed. The mask should be very clear without holes as mentioned by CODA authors.
-  "registered" stores the global registered images
-  "elastic" stores the elastic registered images

In [ ]:
# TODO Run matlab script

In [ ]:
utils.alignment_report("./data/66-4", 32)

Here we can choose over all the trained models ("trained_model" if this directory is empty run the training pipeline to generate a new ML model) saved to predict over a specific set of images. Therefore we need to include also the path from were we will be making the predictions registered, elastic, original etc.

In [ ]:
selected_folder = ""

@interact
def show_files(files=next(os.walk("./trained_models"))[1]):
    global selected_folder
    selected_folder = files
    print("Selected folder:", selected_folder)

Three ways to predict depending on RAM resources:
- predict_stack_untiled() -> Fastest if enough RAM available
- predict_image_stack_tiled() -> Decent time, tiling problem very visible in some models
- predict_image_stack_tiled_smoothed() -> Takes the longest but reduces the tiling effect significantly (also really high RAM dependent)

In [ ]:
model = load_model(os.path.join("trained_models", selected_folder, "unet.h5"), compile=False)
utils.predict_stack_untiled(directory= "./data/66-4/processed_32/clean_dust_bubbles/registered/elastic registration/*.*", model=model, model_trained_size=128)

In [ ]:
# model = load_model(os.path.join("trained_models", "results_20240611_202952", "unet.h5"), compile=False)
# utils.predict_stack_tiled(directory= "./data/66-4/processed_4/clean_dust_bubbles/registered/elastic registration/*.jpg", model=model, model_trained_size=4096)

This will generate a movie in the "videos" directory with a recording of the predicted 3D or the 2D depending on the requirements. 

If you want to inspect the model predictions visually, we recommend to run first 2D. If everything is ok then proceed with an execution of the 3D video.

In [ ]:
# Build 3D
utils.generate_render("./data/66-4/processed_32/clean_dust_bubbles/registered/elastic registration/*.tif"
                      ,"./data/predictions/*.png"
                      , record_video=False
                      , format="3D") # 2D or 3D # This setting only works if record_video is set to true